Reload .py files whenever there are changes

In [2]:
%load_ext autoreload
%autoreload 2

Local imports from this project

In [3]:
from mdf import *
from mdf.data import DataManager

External imports from dependencies

In [4]:
from pytorch3d.ops.laplacian_matrices import norm_laplacian, cot_laplacian, laplacian
from pytorch3d.ops import norm_laplacian, cot_laplacian, laplacian
from pyvista.plotting.plotter import Plotter
import plotly.graph_objects as go
import plotly.express as px
from torch import Tensor
from typing import cast
import torch_geometric
import torch.nn as nn
import pyvista as pv
import scipy.linalg
import torch.linalg
import numpy as np
import pytorch3d
import einops
import torch

Environment setup

In [5]:
pv.set_jupyter_backend('trame')

Data Loading

In [6]:
# Download standard shapes
data = DataManager(data_dir=data_dir, cache_dir=cache_dir)

How to apply a texture to a 3d mesh

In [136]:
vista  = data.objects['woody'].vista
texture = pv.Texture(data.weather[42]['data'])
vista.texture_map_to_sphere(inplace=True)
vista.plot(texture=texture)

/home/invokariman/.cache/pypoetry/virtualenvs/mdf-B-9uJKhD-py3.11/lib/python3.11/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


Widget(value='<iframe src="http://localhost:39485/index.html?ui=P_0x777f122a39d0_96&reconnect=auto" class="pyv…

Compute positional embedding using Laplace-Beltrami Operator

In [137]:
from pytorch3d.structures import Meshes
from torch._tensor import Tensor

# Get vertices and faces
verts: Tensor = torch.tensor(vista.points)
faces: Tensor = torch.tensor(vista.faces.reshape((-1, 4))[:, 1:])

# Eliminate vertices that don't appear in faces
verts_idx: Tensor = torch.arange(verts.size(0))
verts_idx_inuse: Tensor = torch.unique(faces.flatten())
verts_idx_inuse: Tensor = torch.isin(verts_idx, verts_idx_inuse)

# Be able to offset faces properly
offsets = (~verts_idx_inuse).cumsum(0)
offset = np.vectorize(lambda x: x - offsets[x])
faces_inuse = np.array(vista.faces.reshape((-1, 4)))
faces_inuse[:, 1] = offset(faces_inuse[:, 1])
faces_inuse[:, 2] = offset(faces_inuse[:, 2])
faces_inuse[:, 3] = offset(faces_inuse[:, 3])

def get_adjacency_matrix(verts: Tensor, faces: Tensor, index: Tensor) -> Tensor:
    matrix: Tensor = torch.zeros(size=(verts.size(0), verts.size(0)))
    matrix[faces[:, 0], faces[:, 1]] = 1
    matrix[faces[:, 1], faces[:, 0]] = 1
    matrix[faces[:, 0], faces[:, 2]] = 1
    matrix[faces[:, 2], faces[:, 0]] = 1
    matrix[faces[:, 1], faces[:, 2]] = 1
    matrix[faces[:, 2], faces[:, 1]] = 1
    matrix = matrix[index, :][:, index]
    return matrix

def get_degree_matrix(adjacency_matrix: Tensor) -> Tensor:
    matrix = torch.sum(adjacency_matrix, dim=1).to_dense()
    matrix = torch.diag(matrix, diagonal=0)
    return matrix

# Find symmetric normalized graph Laplacian
A: Tensor = get_adjacency_matrix(verts, faces, verts_idx_inuse)
D: Tensor = get_degree_matrix(A)

# Remove unused vertices
verts_inuse: Tensor = verts[verts_idx_inuse, :]

# Compute Laplacian
L = torch.diag(D.diag() ** (-1/2)).to_sparse() @ (D - A).to_sparse() @ torch.diag(D.diag() ** (-1/2)).to_sparse()

In [139]:
# Find the smallest k eigenvalues and their respective eigenvectors
k_eigenvals = 10
val, vec = torch.lobpcg(L.cuda(), k=k_eigenvals, largest=False)

def eigenfunction(vertex_index: int, eigenvector: torch.Tensor) -> torch.Tensor:
    return eigenvector[vertex_index] * torch.sqrt(torch.tensor(eigenvector.size(0)))

def eigen(vertex_index: int, eigenvectors: torch.Tensor) -> torch.Tensor:
    return eigenvectors[vertex_index, :].sum() * torch.sqrt(torch.tensor(eigenvectors.size(0)))

vec = torch.nn.functional.normalize(vec, dim=0)
vec = vec.cpu()

In [141]:
x = np.array([eigen(i, vec[:, 9:10]) for i in range(verts_inuse.shape[0])])
pe = pv.PolyData(verts_inuse.numpy(), faces_inuse)
p = Plotter()
p.add_mesh(pe, scalars=x, clim=[-1, 1], cmap='RdBu_r')
p.show()

Widget(value='<iframe src="http://localhost:39485/index.html?ui=P_0x777f061951d0_98&reconnect=auto" class="pyv…